<a href="https://colab.research.google.com/github/DawidMlynarski/LinearRegression/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()



In [ ]:
X = pd.DataFrame()
dataset.sex = pd.Categorical(dataset.sex)
dataset.smoker = pd.Categorical(dataset.smoker)
dataset.region = pd.Categorical(dataset.region)
region=pd.get_dummies(dataset.region, prefix='location')
dataset = pd.concat([dataset,region],axis=1)
dataset.drop(columns='region', inplace=True)

X['sex'] = dataset.sex.cat.codes
X['smoker'] = dataset.smoker.cat.codes
X['age'] = dataset['age']
X['bmi'] = dataset['bmi']
X['children'] = dataset['children']
X['location_southwest'] = dataset['location_southwest'].astype(int)
X['location_southeast'] = dataset['location_southeast'].astype(int)
X['location_northeast'] = dataset['location_northeast'].astype(int)
X['location_northwest'] = dataset['location_northwest'].astype(int)

features=['age', 'bmi', 'smoker', 'children', 'location_northeast', 'location_northwest', 'location_southeast', 'location_southwest']
kmeans = KMeans(n_clusters=2)
kmeans.fit(X[features])
X['cust_type'] = kmeans.predict(X[features])
X['location_north'] = X.apply(lambda x: 1 if x['location_northeast'] == 1 or x['location_northwest'] == 1 else 0, axis=1)
X['location_south'] = X.apply(lambda x: 1 if x['location_southeast'] == 1 or x['location_southwest'] == 1 else 0, axis=1)
X['more_than_1_child']=X.children.apply(lambda x:1 if x>1 else 0)

X.drop(columns=['location_southeast', 'location_southwest', 'location_northwest', 'location_northeast'], inplace=True)



Y = dataset['expenses']

transformer = preprocessing.StandardScaler().fit(X[['age']])
X['age'] = transformer.transform(X[['age']])
transformer = preprocessing.StandardScaler().fit(X[['bmi']])
X['bmi'] = transformer.transform(X[['bmi']])
transformer = preprocessing.StandardScaler().fit(X[['children']])
X['children'] = transformer.transform(X[['children']])



X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20,random_state=42)


model = DecisionTreeRegressor()
regr_trans = TransformedTargetRegressor(regressor=model, transformer=QuantileTransformer(output_distribution='normal'))
regr_trans.fit(X_train, Y_train)
yhat = regr_trans.predict(X_test)

print('r2 score: ', r2_score(Y_test, yhat))
print(mean_absolute_error(Y_test, yhat))


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
mae = mean_absolute_error(Y_test,yhat)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")


